# 1.04 Find Mixed Layer Ambient Temperature

---

Author : Riley X. Brady

Date : 11/19/20

Here we find the ambient mixed layer temperature for each particle in a given ensemble. To do this, we look at *in situ* particle temperature when it first enters 200 m after its last 1000 m crossing. This ambient temperature is used to calculate potential pCO$_{2}$, the pCO$_{2}$ the particle would have if brought to the point at which it upwells into the mixed layer, due to thermal effects.

In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
import gsw

from dask.distributed import Client

In [2]:
print(f"numpy: {np.__version__}")
print(f"xarray: {xr.__version__}")
print(f"gsw: {gsw.__version__}")

numpy: 1.19.4
xarray: 0.16.1
gsw: 3.4.0


In [3]:
# This is my TCP client from the `launch_cluster` notebook. I use it
# for distributed computing with `dask` on NCAR's machine, Casper.
client = Client("tcp://...")

**Note**: I loaded in the netCDF file, and chunked it, and then saved it back out as a `zarr` file. This makes `dask` run a lot more efficiently. E.g.,

```python
ds = xr.open_dataset('../data/southern_ocean_deep_upwelling_particles.nc')
ds = ds.chunk({'time': -1, 'nParticles': 'auto'})
ds.to_zarr('../data/southern_ocean_deep_upwelling_particles.zarr', consolidated=True)
```

You could probably chunk the particles into slightly smaller chunks for even faster performance.

In [4]:
# Load in the `zarr` file, which is pre-chunked and already has been
# filtered from the original 1,000,000 particles to the 19,002 that
# upwell last across 1000 m S of 45S and outside of the annual sea ice
# edge.
filepath = "../data/southern_ocean_deep_upwelling_particles.zarr/"
ds = xr.open_zarr(filepath, consolidated=True)

In [5]:
def compute_idx_of_last_1000m_crossing(z):
    """Find index of final time particle upwells across 1000 m.

    z : zLevelParticle
    """
    currentDepth = z
    previousDepth = np.roll(z, 1)
    previousDepth[0] = 999  # So we're not dealing with a nan here.
    cond = (currentDepth >= -1000) & (previousDepth < -1000)
    idx = (
        len(cond) - np.flip(cond).argmax() - 1
    )  # Finds last location that condition is true.
    return idx


def compute_idx_of_first_200m_crossing(z):
    """Find first time particle upwells across 200 m.

    z : zLevelParticle
    """
    currentDepth = z
    previousDepth = np.roll(z, 1)
    previousDepth[0] = 999
    cond = (currentDepth >= -200) & (previousDepth < -200)
    idx = cond.argmax()
    return idx


def find_200m_ambient_temperature(z, t):
    """
    Finds the ambient temperature at the 200m crossing point.
    """
    idx_a = compute_idx_of_last_1000m_crossing(z)
    z_subset = z[idx_a::]
    idx_b = compute_idx_of_first_200m_crossing(z_subset)
    return t[idx_a + idx_b]

## Calculate ambient temperature for each ensemble

In [6]:
for region_name in ["drake", "crozet", "campbell", "kerguelan", "non_topographic"]:
    print(f"{region_name}...")
    # Load in ensemble for given region to get particle IDs.
    ensemble_ids = xr.open_dataset(
        f"../data/postproc/{region_name}.1000m.tracer.origin.nc"
    )
    ensemble = ds.sel(nParticles=ensemble_ids.nParticles)

    # Calculate in situ temperature, since the particle temperature is actually
    # potential temperature.
    ensemble["t_insitu"] = gsw.pt_from_t(
        ensemble.particleSalinity,
        ensemble.particleTemperature,
        0,
        ensemble.zLevelParticle * -1,
    )

    # Calculate ambient temperature for the given ensemble.
    ambient = xr.apply_ufunc(
        find_200m_ambient_temperature,
        ensemble.zLevelParticle,
        ensemble.t_insitu,
        input_core_dims=[["time"], ["time"]],
        dask="parallelized",
        vectorize=True,
        output_dtypes=[float],
        dask_gufunc_kwargs={"allow_rechunk": True},
    )
    %time ambient = ambient.compute()

    print("Saving to netCDF...")
    ds_out = ambient.rename("T_ambient").to_dataset()
    ds_out.attrs[
        "description"
    ] = "in situ particle temperature when particle first enters mixed layer (200 m) after its last 1000 m crossing."
    ds_out.to_netcdf(f"../data/postproc/{region_name}.ambient.temperature.nc")

drake...
CPU times: user 11.1 ms, sys: 4.05 ms, total: 15.2 ms
Wall time: 2.71 s
Saving to netCDF...
crozet...
CPU times: user 16.3 ms, sys: 715 µs, total: 17 ms
Wall time: 4.62 s
Saving to netCDF...
campbell...
CPU times: user 14.6 ms, sys: 179 µs, total: 14.7 ms
Wall time: 2.22 s
Saving to netCDF...
kerguelan...
CPU times: user 14.6 ms, sys: 0 ns, total: 14.6 ms
Wall time: 1.87 s
Saving to netCDF...
non_topographic...
CPU times: user 14.2 ms, sys: 1.07 ms, total: 15.3 ms
Wall time: 2.65 s
Saving to netCDF...
